In [185]:
import pandas as pd
import numpy as np
import nltk
from string import punctuation

from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/nickgeissler/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/nickgeissler/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/nickgeissler/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [186]:
# Check location of the data
df1 = pd.read_csv("data/spam.csv", usecols=[0,1])
df1['Category'] = df1['Category'].replace({'ham': 0, 'spam': 1})

df2 = pd.read_csv("data/enronSpamSubset.csv", usecols=[1,0])
df3 = pd.read_csv("data/lingSpam.csv", usecols=[1,0])
df4 = pd.read_csv("data/completeSpamAssassin.csv", usecols=[1,0])

print(df2.shape, df3.shape, df4.shape)
df2.columns = df1.columns
df3.columns = df1.columns
df4.columns = df1.columns

allData = pd.concat([df1, df2, df3, df4])
allData = allData[pd.to_numeric(allData['Category'], errors='coerce').notna()]
allData = allData.dropna()
allData.iloc[:, 0] = allData.iloc[:, 0].astype(int)
allData = allData[allData['Category'].isin([0, 1])]

allData
#print(df3, df4)

/var/folders/sc/hq14ct8s2cj37gzs0drpl0x00000gn/T/ipykernel_2112/273821164.py:5: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df2 = pd.read_csv("data/enronSpamSubset.csv", usecols=[1,0])


(16146, 2) (23705, 2) (22409, 2)


/var/folders/sc/hq14ct8s2cj37gzs0drpl0x00000gn/T/ipykernel_2112/273821164.py:6: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df3 = pd.read_csv("data/lingSpam.csv", usecols=[1,0])
/var/folders/sc/hq14ct8s2cj37gzs0drpl0x00000gn/T/ipykernel_2112/273821164.py:7: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df4 = pd.read_csv("data/completeSpamAssassin.csv", usecols=[1,0])
/var/folders/sc/hq14ct8s2cj37gzs0drpl0x00000gn/T/ipykernel_2112/273821164.py:17: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  allData.iloc[:, 0] = allData.iloc[:, 0].astype(int)


,Category,Message
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
239,1,\nDear Sir or MadamIn the past you have reques...
240,1,empty
241,1,\nDear Sir or MadamIn the past you have reques...
242,1,********************...


In [187]:
print(allData['Message'][0])

0    Go until jurong point, crazy.. Available only ...
0    Subject: stock promo mover : cwtd\n * * * urge...
0    Subject: great part-time or summer job !\n \n ...
0    \nSave up to 70% on Life Insurance.\nWhy Spend...
Name: Message, dtype: object


In [188]:
print("Count of label:\n",allData['Category'].value_counts())

Count of label:
 0    11976
1     6424
Name: Category, dtype: int64


PREPROCESSING

In [189]:
import string
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [190]:
def remove_punc(text):
    nonP_text = "".join([char for char in text if char not in string.punctuation])
    return nonP_text

In [191]:
allData['no_punc'] = allData['Message'].apply(lambda x: remove_punc(x))
allData

,Category,Message,no_punc
0,0,"Go until jurong point, crazy.. Available only ...",Go until jurong point crazy Available only in ...
1,0,Ok lar... Joking wif u oni...,Ok lar Joking wif u oni
2,1,Free entry in 2 a wkly comp to win FA Cup fina...,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...,U dun say so early hor U c already then say
4,0,"Nah I don't think he goes to usf, he lives aro...",Nah I dont think he goes to usf he lives aroun...
...,...,...,...
239,1,\nDear Sir or MadamIn the past you have reques...,\nDear Sir or MadamIn the past you have reques...
240,1,empty,empty
241,1,\nDear Sir or MadamIn the past you have reques...,\nDear Sir or MadamIn the past you have reques...
242,1,********************...,\n ...


In [192]:
import re

In [193]:
def tokenize(text):
    tokens = re.split("\W+", text)
    return tokens

In [194]:
allData['tokenized'] = allData['no_punc'].apply(lambda x: tokenize(x))
allData

,Category,Message,no_punc,tokenized
0,0,"Go until jurong point, crazy.. Available only ...",Go until jurong point crazy Available only in ...,"[Go, until, jurong, point, crazy, Available, o..."
1,0,Ok lar... Joking wif u oni...,Ok lar Joking wif u oni,"[Ok, lar, Joking, wif, u, oni]"
2,1,Free entry in 2 a wkly comp to win FA Cup fina...,Free entry in 2 a wkly comp to win FA Cup fina...,"[Free, entry, in, 2, a, wkly, comp, to, win, F..."
3,0,U dun say so early hor... U c already then say...,U dun say so early hor U c already then say,"[U, dun, say, so, early, hor, U, c, already, t..."
4,0,"Nah I don't think he goes to usf, he lives aro...",Nah I dont think he goes to usf he lives aroun...,"[Nah, I, dont, think, he, goes, to, usf, he, l..."
...,...,...,...,...
239,1,\nDear Sir or MadamIn the past you have reques...,\nDear Sir or MadamIn the past you have reques...,"[, Dear, Sir, or, MadamIn, the, past, you, hav..."
240,1,empty,empty,[empty]
241,1,\nDear Sir or MadamIn the past you have reques...,\nDear Sir or MadamIn the past you have reques...,"[, Dear, Sir, or, MadamIn, the, past, you, hav..."
242,1,********************...,\n ...,"[, ï, ½ï, ½ï, ½ï, ½ï, ½ï, ½ï, ½ï, ½ï, ½ï, ½ï, ..."


In [195]:
import nltk
stopwords = nltk.corpus.stopwords.words("english")

In [196]:
def remove_stopwords(token):
    text = [word for word in token if word not in stopwords]# to remove all stopwords
    return text

In [197]:
allData['nonstop'] = allData['tokenized'].apply(lambda x: remove_stopwords(x))
allData

,Category,Message,no_punc,tokenized,nonstop
0,0,"Go until jurong point, crazy.. Available only ...",Go until jurong point crazy Available only in ...,"[Go, until, jurong, point, crazy, Available, o...","[Go, jurong, point, crazy, Available, bugis, n..."
1,0,Ok lar... Joking wif u oni...,Ok lar Joking wif u oni,"[Ok, lar, Joking, wif, u, oni]","[Ok, lar, Joking, wif, u, oni]"
2,1,Free entry in 2 a wkly comp to win FA Cup fina...,Free entry in 2 a wkly comp to win FA Cup fina...,"[Free, entry, in, 2, a, wkly, comp, to, win, F...","[Free, entry, 2, wkly, comp, win, FA, Cup, fin..."
3,0,U dun say so early hor... U c already then say...,U dun say so early hor U c already then say,"[U, dun, say, so, early, hor, U, c, already, t...","[U, dun, say, early, hor, U, c, already, say]"
4,0,"Nah I don't think he goes to usf, he lives aro...",Nah I dont think he goes to usf he lives aroun...,"[Nah, I, dont, think, he, goes, to, usf, he, l...","[Nah, I, dont, think, goes, usf, lives, around..."
...,...,...,...,...,...
239,1,\nDear Sir or MadamIn the past you have reques...,\nDear Sir or MadamIn the past you have reques...,"[, Dear, Sir, or, MadamIn, the, past, you, hav...","[, Dear, Sir, MadamIn, past, requested, inform..."
240,1,empty,empty,[empty],[empty]
241,1,\nDear Sir or MadamIn the past you have reques...,\nDear Sir or MadamIn the past you have reques...,"[, Dear, Sir, or, MadamIn, the, past, you, hav...","[, Dear, Sir, MadamIn, past, requested, inform..."
242,1,********************...,\n ...,"[, ï, ½ï, ½ï, ½ï, ½ï, ½ï, ½ï, ½ï, ½ï, ½ï, ½ï, ...","[, ï, ½ï, ½ï, ½ï, ½ï, ½ï, ½ï, ½ï, ½ï, ½ï, ½ï, ..."


In [198]:
ps = nltk.SnowballStemmer("english",ignore_stopwords = True)

def stemming(t_text):
    text = [ps.stem(word) for word in t_text]
    return text

allData['stemmed'] = allData['nonstop'].apply(lambda x: stemming(x))
allData

,Category,Message,no_punc,tokenized,nonstop,stemmed
0,0,"Go until jurong point, crazy.. Available only ...",Go until jurong point crazy Available only in ...,"[Go, until, jurong, point, crazy, Available, o...","[Go, jurong, point, crazy, Available, bugis, n...","[go, jurong, point, crazi, avail, bugi, n, gre..."
1,0,Ok lar... Joking wif u oni...,Ok lar Joking wif u oni,"[Ok, lar, Joking, wif, u, oni]","[Ok, lar, Joking, wif, u, oni]","[ok, lar, joke, wif, u, oni]"
2,1,Free entry in 2 a wkly comp to win FA Cup fina...,Free entry in 2 a wkly comp to win FA Cup fina...,"[Free, entry, in, 2, a, wkly, comp, to, win, F...","[Free, entry, 2, wkly, comp, win, FA, Cup, fin...","[free, entri, 2, wkli, comp, win, fa, cup, fin..."
3,0,U dun say so early hor... U c already then say...,U dun say so early hor U c already then say,"[U, dun, say, so, early, hor, U, c, already, t...","[U, dun, say, early, hor, U, c, already, say]","[u, dun, say, earli, hor, u, c, alreadi, say]"
4,0,"Nah I don't think he goes to usf, he lives aro...",Nah I dont think he goes to usf he lives aroun...,"[Nah, I, dont, think, he, goes, to, usf, he, l...","[Nah, I, dont, think, goes, usf, lives, around...","[nah, i, dont, think, goe, usf, live, around, ..."
...,...,...,...,...,...,...
239,1,\nDear Sir or MadamIn the past you have reques...,\nDear Sir or MadamIn the past you have reques...,"[, Dear, Sir, or, MadamIn, the, past, you, hav...","[, Dear, Sir, MadamIn, past, requested, inform...","[, dear, sir, madamin, past, request, inform, ..."
240,1,empty,empty,[empty],[empty],[empti]
241,1,\nDear Sir or MadamIn the past you have reques...,\nDear Sir or MadamIn the past you have reques...,"[, Dear, Sir, or, MadamIn, the, past, you, hav...","[, Dear, Sir, MadamIn, past, requested, inform...","[, dear, sir, madamin, past, request, inform, ..."
242,1,********************...,\n ...,"[, ï, ½ï, ½ï, ½ï, ½ï, ½ï, ½ï, ½ï, ½ï, ½ï, ½ï, ...","[, ï, ½ï, ½ï, ½ï, ½ï, ½ï, ½ï, ½ï, ½ï, ½ï, ½ï, ...","[, ï, ½ï, ½ï, ½ï, ½ï, ½ï, ½ï, ½ï, ½ï, ½ï, ½ï, ..."


In [199]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk import word_tokenize, pos_tag
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')

# From https://www.ibm.com/topics/stemming-lemmatization 
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:         
        return wordnet.NOUN
       
def lemmatize_email(words):
    pos_tags = pos_tag(words)
    lemmatizer = WordNetLemmatizer()
    lemmatized_words = [lemmatizer.lemmatize(word, get_wordnet_pos(tag)) for word, tag in pos_tags]
    return lemmatized_words

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/nickgeissler/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/nickgeissler/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [200]:
#allData['cleaned_msg'] = allData['stemmed'].apply(lambda x: lemmatizer(x))
allData['cleaned_msg'] = allData['no_punc'].apply(lambda x: lemmatize_email(x))
allData

<class 'str'>
<class 'list'>
<class 'str'>
<class 'list'>
<class 'str'>
<class 'list'>
<class 'str'>
<class 'list'>
<class 'str'>
<class 'list'>
<class 'str'>
<class 'list'>
<class 'str'>
<class 'list'>
<class 'str'>
<class 'list'>
<class 'str'>
<class 'list'>
<class 'str'>
<class 'list'>
<class 'str'>
<class 'list'>
<class 'str'>
<class 'list'>
<class 'str'>
<class 'list'>
<class 'str'>
<class 'list'>
<class 'str'>
<class 'list'>
<class 'str'>
<class 'list'>
<class 'str'>
<class 'list'>
<class 'str'>
<class 'list'>
<class 'str'>
<class 'list'>
<class 'str'>
<class 'list'>
<class 'str'>
<class 'list'>
<class 'str'>
<class 'list'>
<class 'str'>
<class 'list'>
<class 'str'>
<class 'list'>
<class 'str'>
<class 'list'>
<class 'str'>
<class 'list'>
<class 'str'>
<class 'list'>
<class 'str'>
<class 'list'>
<class 'str'>
<class 'list'>
<class 'str'>
<class 'list'>
<class 'str'>
<class 'list'>
<class 'str'>
<class 'list'>
<class 'str'>
<class 'list'>
<class 'str'>
<class 'list'>
<class 'str'>


IndexError: list index out of range

In [ ]:
cleanData = allData[['Category', 'cleaned_msg']].copy()
cleanData

,Category,cleaned_msg
0,0,"[Go, until, jurong, point, crazy, Available, o..."
1,0,"[Ok, lar, Joking, wif, u, oni]"
2,1,"[Free, entry, in, 2, a, wkly, comp, to, win, F..."
3,0,"[U, dun, say, so, early, hor, U, c, already, t..."
4,0,"[Nah, I, dont, think, he, go, to, usf, he, liv..."
...,...,...
239,1,"[Dear, Sir, or, MadamIn, the, past, you, have,..."
240,1,[empty]
241,1,"[Dear, Sir, or, MadamIn, the, past, you, have,..."
242,1,"[ï¿½ï¿½ï¿½ï¿½ï¿½ï¿½ï¿½ï¿½ï¿½ï¿½ï¿½ï¿½Xï¿½ï¿½, ..."


In [ ]:
# eliminate duplicates within the same email
cleanData['nondupe'] = cleanData['cleaned_msg'].apply(lambda x: list(set(x)))
flat = cleanData['nondupe'].explode()

# only want words that appear in multiple emails
def find_multiple_appearances(strings):
    string_count = {}
    duplicates_set = set()

    # Count occurrences of each string
    for string in strings:
        string_count[string] = string_count.get(string, 0) + 1

    # Add strings with counts >= 2 to the duplicates set
    for string, count in string_count.items():
        if count >= 2:
            duplicates_set.add(string)

    return duplicates_set
    
allStrings = find_multiple_appearances(flat)
len(allStrings)

56692

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

cleanData['cleaned_msg'] = cleanData['cleaned_msg'].apply(' '.join)

# Initialize CountVectorizer with your specific vocabulary
vectorizer = CountVectorizer(vocabulary=allStrings)

# Fit and transform the 'cleaned_msg' column
X = vectorizer.fit_transform(cleanData['cleaned_msg'])

# Convert the result to a DataFrame and add it to the original DataFrame
count_df = pd.DataFrame.sparse.from_spmatrix(X, columns=vectorizer.get_feature_names_out())

# Concatenate the sparse DataFrame with the original DataFrame
cleanData = cleanData.reset_index(drop=True)
count_df = count_df.reset_index(drop=True)
cleanData = pd.concat([cleanData, count_df], axis=1)


TypeError: '<' not supported between instances of 'float' and 'str'

In [ ]:
sum(cleanData.iloc[:, 3])

0

In [ ]:
# None of this shit works - for now, we can just use a smaller dataset to test the rest of the code

# import dask.dataframe as dd 
# #cleanData.to_csv('data/cleanData.csv', index=False) 

# # Pandas DataFrame to a Dask DataFrame
# dask_df = dd.from_pandas(cleanData, npartitions=3)
# dask_df.to_parquet("data/cleanData.parquet", compression="gzip")

